<a href="https://colab.research.google.com/github/specM7/DSGP_Group_33_Brain_Tumor_Predictor/blob/main/Meningioma_Adrian_2425482.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧠 AI-Powered Brain Tumor Predictor for Meningioma(Adrian)


In [8]:
# Core
import os, glob
import numpy as np

# Image processing
import cv2
from google.colab.patches import cv2_imshow

# Deep learning
import tensorflow as tf
from tensorflow.keras import layers, models

# Visualization
import matplotlib.pyplot as plt


In [9]:
from google.colab import files
uploaded = files.upload()          # choose DataSet.zip from your PC

!unzip DataSet.zip -d /content
!ls /content/DataSet


Saving DataSet.zip to DataSet (1).zip
Archive:  DataSet.zip
error [DataSet.zip]:  missing 54525952 bytes in zipfile
  (attempting to process anyway)
error: invalid zip file with overlapped components (possible zip bomb)
ls: cannot access '/content/DataSet': No such file or directory


In [11]:
def preprocess_img(path, size=(224, 224)):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)        # read MRI
    img = cv2.resize(img, size)

    # noise removal
    img = cv2.GaussianBlur(img, (5, 5), 0)

    # contrast enhancement
    img = cv2.equalizeHist(img)

    # normalize 0–1
    img = img.astype("float32") / 255.0
    return img
